In [6]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
from pathlib import Path

#import api_key
#from api_keys import api_keys

In [7]:
#Createing path to CSV files
Flu_path = Path("../Resources/From national centers for enviromental info2016-2019.csv")
Temp_path = Path("../Resources/state_temperatures.csv")


In [3]:
#Create data frame
Flu_DF = pd.read_csv(Flu_path)
Temp_DF = pd.read_csv(Temp_path)

Flu_DF.head()
Temp_DF.head()

,State,Year,Month,Week,AverageTemperature
0,Alabama,2016,1.0,53.0,44.565887
1,Alabama,2016,1.0,1.0,45.227760
2,Alabama,2016,1.0,2.0,43.475330
3,Alabama,2016,1.0,3.0,39.243889
4,Alabama,2016,1.0,4.0,48.005981


In [4]:
Flu_DF.head()

,Southeast Climate Region,Average Temperature
0,Units: Degrees Fahrenheit,NaN
1,Missing: -99,NaN
2,Date,Value
3,201601,43.8
4,201602,48.8


In [5]:
# Remove NaN from Temp_DF
Temp_DF.head().dropna()

,State,Year,Month,Week,AverageTemperature
0,Alabama,2016,1.0,53.0,44.565887
1,Alabama,2016,1.0,1.0,45.227760
2,Alabama,2016,1.0,2.0,43.475330
3,Alabama,2016,1.0,3.0,39.243889
4,Alabama,2016,1.0,4.0,48.005981
